In [47]:

import pandas as pd
import numpy as np
from typing import Dict, Any
from datetime import datetime
import numpy as np
from extract.extract_new_tokens import make_request
import logging
# Now you can use absolute imports
from utils.clean_numeric_columns import clean_numeric_columns
from utils.convert_boolean_columns import convert_boolean_columns
from utils.flatten_json import flatten_json
import json

In [69]:
import pandas as pd
from decimal import Decimal, InvalidOperation

def clean_numeric_columns(df, numeric_cols):
    """Converts numeric columns to Decimal for precision without worrying about trailing zeros."""
    for col in numeric_cols:
        if col in df.columns:
            # Convert to Decimal for high precision
            def convert_value(x):
                try:
                    if pd.isna(x) or str(x).strip() in ['', 'None', 'nan']:
                        return None  # Use None for missing values
                    return Decimal(str(x))  # Convert to Decimal
                except (InvalidOperation, TypeError, ValueError):
                    return None  # Handle invalid inputs
            
            df[col] = df[col].apply(convert_value)
    
    return df

In [72]:

raw_data = make_request()
with open("has_zero.json", "w", encoding="utf-8") as f:
    json.dump(raw_data, f, indent=4)
pd.set_option('future.no_silent_downcasting', True)

def transform_new_tokens(json_data: dict) -> pd.DataFrame:
    """Transforms and cleans GMGN JSON data into a properly typed DataFrame."""
    logging.info("Starting data transformation...")
    
    # Early return if no data
    if not json_data or not json_data.get('data', {}).get('pairs'):
        logging.warning("No data found in JSON input.")
        return pd.DataFrame()

    # Extract and flatten the raw data
    raw_data = json_data["data"]["pairs"]
    flat_records = [flatten_json(item) for item in raw_data]
    df = pd.DataFrame(flat_records)
    
    # Drop unnecessary columns
    cols_to_drop = [
        'id', 'pool_type', 'quote_address', 'base_token_info_launchpad_status',
        'base_token_info_buy_tax', 'base_token_info_sell_tax', 'base_token_info_is_honeypot',
        'base_token_info_renounced', 'base_token_info_dexscr_ad', 'base_token_info_dexscr_update_link',
        'base_token_info_is_open_source', 'base_token_info_lockInfo', 'base_token_info_progress'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    # Column combinations and filling missing values
    column_combinations = [
        ('pool_type_str', 'launchpad'),
        ('base_address', 'base_token_info_address'),
        ('base_token_info_burn_status', 'burn_status'),
        ('base_token_info_burn_ratio', 'burn_ratio'),
        ('base_token_info_liquidity', 'liquidity'),
        ('address', 'base_token_info_pool_id'),
        ('creation_timestamp', 'base_token_info_creation_timestamp')
    ]
    
    for new_col, old_col in column_combinations:
        if new_col in df.columns and old_col in df.columns:
            df[new_col] = df[new_col].fillna(df[old_col])
            df.drop(columns=[old_col], inplace=True)
        elif new_col not in df.columns and old_col in df.columns:
            df[new_col] = df[old_col]
            df.drop(columns=[old_col], inplace=True)
    
    # Column renaming
    rename_map = {
        'address': 'pair_address',
        'base_address': 'address',
        'pool_type_str': 'platform',
        'base_token_info_symbol': 'symbol',
        'base_token_info_name': 'name',
        'base_token_info_logo': 'logo',
        'base_token_info_total_supply': 'total_supply',
        'base_token_info_holder_count': 'holder_count',
        'base_token_info_sniper_count': 'sniper_count',
        'base_token_info_price_change_percent1m': 'price_change_1m',
        'base_token_info_price_change_percent5m': 'price_change_5m',
        'base_token_info_price_change_percent1h': 'price_change_1h',
        'base_token_info_price': 'price',
        'base_token_info_is_show_alert': 'has_alert',
        'base_token_info_hot_level': 'hot_level',
        'base_token_info_liquidity': 'liquidity',
        'base_token_info_top_10_holder_rate': 'top_10_holder_rate',
        'base_token_info_renounced_mint': 'renounced_mint',
        'base_token_info_renounced_freeze_account': 'renounced_freeze_account',
        'base_token_info_social_links_twitter_username': 'twitter_username',
        'base_token_info_social_links_website': 'website',
        'base_token_info_social_links_telegram': 'telegram',
        'base_token_info_rug_ratio': 'rug_ratio',
        'base_token_info_is_wash_trading': 'is_wash_trading',
        'base_token_info_creator_balance_rate': 'creator_balance_rate',
        'base_token_info_rat_trader_amount_rate': 'rat_trader_amount_rate',
        'base_token_info_creator_token_status': 'creator_token_status',
        'base_token_info_bluechip_owner_percentage': 'bluechip_owner_percentage',
        'base_token_info_smart_degen_count': 'smart_degen_count',
        'base_token_info_renowned_count': 'renowned_count',
        'base_token_info_volume': 'volume',
        'base_token_info_swaps': 'swaps',
        'base_token_info_buys': 'buys',
        'base_token_info_sells': 'sells',
        'base_token_info_burn_status': 'burn_status',
        'base_token_info_burn_ratio': 'burn_ratio',
        'base_token_info_dev_token_burn_amount': 'dev_token_burn_amount',
        'base_token_info_dev_token_burn_ratio': 'dev_token_burn_ratio',
        'base_token_info_cto_flag': 'cto_flag',
        'base_token_info_twitter_change_flag': 'twitter_change_flag',
        'base_token_info_market_cap': 'market_cap',
        'base_token_info_creator_close': 'creator_close',
        'base_token_info_biggest_pool_address': 'biggest_pool_address'
    }
    
    existing_rename_map = {k: v for k, v in rename_map.items() if k in df.columns}
    df.rename(columns=existing_rename_map, inplace=True)

    # Timestamp conversion
    timestamp_cols = ['open_timestamp', 'creation_timestamp']
    for col in timestamp_cols:
        if col in df.columns:
            # First try UNIX timestamp, then string format
            try:
                df[col] = pd.to_datetime(df[col], unit='s', errors='coerce')
            except:
                df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Clean numeric columns
    numeric_cols = [
        'price', 'liquidity', 'volume', 'market_cap', 'quote_reserve',
        'initial_liquidity', 'initial_quote_reserve', 'total_supply',
        'holder_count', 'sniper_count', 'hot_level', 'top_10_holder_rate',
        'rug_ratio', 'rat_trader_amount_rate', 'bluechip_owner_percentage',
        'smart_degen_count', 'renowned_count', 'swaps', 'buys', 'sells',
        'buy_tax', 'sell_tax', 'dev_token_burn_amount', 'dev_token_burn_ratio',
           'bot_degen_count', 'launchpad_status'
    ]
    df = clean_numeric_columns(df, numeric_cols)
    
    # Convert boolean columns
    bool_cols = [
        'has_alert', 'is_wash_trading', 
        'renounced_mint', 'renounced_freeze_account', 'creator_close', 'cto_flag','twitter_change_flag'
    ]
    bool_map = {
        'true': True, 'false': False,
        '1': True, '0': False,
        'yes': True, 'no': False,
        't': True, 'f': False
    }
    df = convert_boolean_columns(df, bool_cols, bool_map)
    
    # Ensure address is clean
    if 'address' in df.columns:
        df['address'] = df['address'].str.strip()
        df = df[df['address'].notna()]
    
    # Add status column with default
    df['status'] = 'alive'
    
    # Final cleanup - replace NaN/NaT with appropriate values
    df = df.replace({np.nan: None, pd.NaT: None})
    
    # Validate and handle missing values in required columns
    required_cols = ['address', 'status', 'symbol', 'platform']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    for col in required_cols:
        if col in df.columns:
            if df[col].dtype == 'object':  # String/object columns
                df[col] = df[col].fillna("")
            elif df[col].dtype == 'bool':  # Boolean columns
                df[col] = df[col].fillna(False)
            else:  # Numeric columns
                df[col] = df[col].fillna(0)
    
    logging.info(f"Transformed DataFrame shape: {df.shape}")
    logging.info(f"Columns in transformed DataFrame: {list(df.columns)}")
    
    return df
df = transform_new_tokens(raw_data)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows

df


2025-04-10 07:23:18,817 - DEBUG - GETCONF __owningcontrollerprocess (runtime: 0.0001)
2025-04-10 07:23:18,828 - INFO - Successfully renewed Tor circuit
2025-04-10 07:23:24,471 - INFO - Error while receiving a control message (SocketClosed): received exception "read of closed file"
2025-04-10 07:23:24,472 - INFO - Using Tor IP: 171.25.193.235
2025-04-10 07:23:24,609 - INFO - HTTP Status Code: 403
2025-04-10 07:23:24,609 - WARNING - Received 403 Forbidden
2025-04-10 07:23:24,610 - INFO - Waiting 10.8s before retry...
2025-04-10 07:23:35,450 - DEBUG - GETCONF __owningcontrollerprocess (runtime: 0.0002)
2025-04-10 07:23:35,456 - INFO - Successfully renewed Tor circuit
2025-04-10 07:23:40,915 - INFO - Error while receiving a control message (SocketClosed): received exception "read of closed file"
2025-04-10 07:23:40,916 - INFO - Using Tor IP: 192.42.116.186
2025-04-10 07:23:41,561 - INFO - HTTP Status Code: 200
2025-04-10 07:23:41,568 - INFO - Filtered 90 new tokens from 100 total tokens
20

,pair_address,address,creation_timestamp,quote_symbol,open_timestamp,quote_reserve,initial_liquidity,initial_quote_reserve,quote_reserve_usd,name,symbol,logo,total_supply,creator_close,creator_token_status,burn_ratio,burn_status,biggest_pool_address,has_alert,hot_level,renounced_mint,renounced_freeze_account,dev_token_burn_amount,dev_token_burn_ratio,cto_flag,twitter_change_flag,liquidity,rug_ratio,top_10_holder_rate,creator_balance_rate,price_change_1m,price_change_5m,price_change_1h,holder_count,sniper_count,smart_degen_count,renowned_count,rat_trader_amount_rate,bluechip_owner_percentage,market_cap,is_wash_trading,volume,swaps,buys,sells,creator,platform,price,bot_degen_count,twitter_username,website,telegram,status
0,DKjkaRoAfR4dQ6vDbSb8uZfM9Q4qiRwwqddGQTTLoBvy,FGp8xjmZnGBFB6vAMNewjJazSHqAhJfozyWbRJ88nqTv,2025-04-10 06:23:40,SOL,2025-04-10 06:23:40,0.01512,3.5295,0.01512,1.7646,A,A,None,1000000000,False,creator_hold,1.0000,burn,DKjkaRoAfR4dQ6vDbSb8uZfM9Q4qiRwwqddGQTTLoBvy,False,0,True,True,None,None,False,False,3.5295,None,None,None,None,None,None,1,0,0,0,0,0.0,None,False,None,None,None,None,None,Pump.fun,None,None,None,None,None,alive
1,FXSmUGzwUjhE33e6usuAb3Qz7W5R8YG3yspbmaawjDd7,BxPDT6RhMK4BVYfU4TNgKh9AcSo7pXm3G2qrKY8Tpump,2025-04-10 06:23:39,SOL,2025-04-10 06:23:39,0.01512,3.5295,0.01512,1.7647,ELON JOCKEY,ELONJOCKEY,None,1000000000,False,creator_hold,1.0000,burn,FXSmUGzwUjhE33e6usuAb3Qz7W5R8YG3yspbmaawjDd7,False,0,True,True,None,None,False,False,3.5295,None,None,None,None,None,None,1,0,0,0,0,0.0,None,False,None,None,None,None,None,Pump.fun,None,None,None,None,None,alive
2,7EgoWR58ZxXNb3mHrXJY7rwUCRWZjVAAx5kkQuBGPUsd,kgxtcMDXZ7FCNu1xzo9pgNzXZTDFyCY1eCHCvDFpump,2025-04-10 06:23:38,SOL,2025-04-10 06:23:38,0.01512,3.5298,0.01512,1.7647,BlackAmerican,BLACK,None,1000000000,False,creator_hold,1.0000,burn,7EgoWR58ZxXNb3mHrXJY7rwUCRWZjVAAx5kkQuBGPUsd,False,0,True,True,None,None,False,False,3.5298,None,None,None,None,None,None,1,0,0,0,0,0.0,None,False,None,None,None,None,None,Pump.fun,None,None,None,None,None,alive
3,9kQrT84osVBE1gNb2WV2EcanSssAK31feuujTEkCDJj2,7JVTaN8wjyk7ebavaovh9TytA6FdhFLUybpn5StA1rUZ,None,SOL,2025-04-10 06:23:30,15.0203,3500.4,15,None,Found The Retard,FTR,https://gmgn.ai/external-res/c5332b39219b198a8...,1000000000,False,creator_add_liquidity,,unknown,None,True,0,True,True,None,None,False,False,3507.239,0.9608728367193379,0.117078,0.02946,0.3335,0.3335,0.3335,162,0,0,0,0,0.0,8797.8469,False,2.3926,7.0,7.0,0.0,8Jf1yfRwHUsn9ULqwsaSAuoW55QHfAMLhZ1LyxSmXfWv,pumpamm,0.000008798,None,None,None,None,alive
4,GYF2d9cQeKsFWF3YbJNvwh6yFYCpFdkDe3mh9oxHjCen,5BWSa7dCt7DKUqexSUagJtE2a2fR4CqxwnY4Vbuqpump,None,SOL,2025-04-10 06:23:27,20.06435,3.5268,0.01512,None,PissCoin,PissCoin,https://pump.mypinata.cloud/ipfs/QmaCtybVah3JR...,1000000000,False,creator_hold,1.0000,burn,None,True,0,True,True,None,None,False,False,728.1706,None,0.246094,0.03461,184.0379,184.0379,184.0379,46,27,0,0,0,0.0,9573.2683,False,3032.4517,58.0,51.0,7.0,None,pump,0.000009573,24,pisscoin_solana,https://x.com/pisscoin_solana,,alive
5,7KWcDcj3NCDPraRh1u8eeoAD5GFo1rCUufb324ES3mzt,6ZCLWBombWuKx2BkhAdyTfVnHhNyH6Hazk2HWjVPpump,None,SOL,2025-04-10 06:23:25,9.9737,3.5271,0.01512,None,amazon_dot_com ELONS CHARACTER,ADC,https://pump.mypinata.cloud/ipfs/QmWx7NfXLfRC2...,1000000000,False,creator_hold,1.0000,burn,None,True,0,True,True,None,None,False,False,2328.2257,None,0.267623,0.06706,66.5436,66.5436,66.5436,10,7,0,0,0,0.0,5794.7869,False,1165.2872,11.0,10.0,1.0,None,pump,0.000005795,1,i/broadcasts/1vAxRDpYmgDGl,,,alive
6,D35Lyupsm36LrCCHGPsp6Eu6iDXkBqgRauQNLZE91USe,4G2pXfR1UyrpjzACGeGGEP8msP2tC94eCH7gDPeepump,None,SOL,2025-04-10 06:23:22,0.5533,3.5274,0.01512,None,Dat My Best Friend,BF,https://gmgn.ai/external-res/5d328a83ed4e41eaa...,1000000000,False,creator_hold,1.0000,burn,None,True,0,True,True,None,None,False,False,129.1214,1.0,0.0193893,0.01759,1.99,1.99,1.99,4,0,0,0,0,0.0,3382.6276,False,64.4169,3.0,3.0,0.0,None,pump,0.000003383,0,,https://www.cbp.gov/doc